In [1]:
import sys
import cv2
import os
import random
import numpy as np

import tensorflow as tf
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import sys
from PIL import Image
sys.modules['Image'] = Image

import matplotlib.pyplot as plt
import imageutils as imageutils
%matplotlib inline

assert (sys.version_info[0]>=3 and sys.version_info[1]>=6), "Wrong Python Version. Please use >3.6"

Using TensorFlow backend.


#### Load Images and initialize Data-Augmented Data-Loaders

In [ ]:
img_width, img_height = 112, 112

train_data_dir = "./data/slices/train"
validation_data_dir = "./data/slices/validation"

batch_size = 32
epochs = 1000

In [ ]:
trainDatagenerator = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = [0.0, 0.2],
    rescale = 1./255)

testDatagenerator = ImageDataGenerator(rescale = 1./255)

trainGenerator = trainDatagenerator.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
shuffle=True,
class_mode = "categorical")

validationGenerator = testDatagenerator.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

nb_train_samples = trainGenerator.n
nb_validation_samples = validationGenerator.n

In [ ]:
validationGenerator.class_indices

#### Initialize Model (VGG19 for initial test - should switch to Resnet later...)

In [ ]:
model = applications.VGG19(weights = None, include_top=False, input_shape = (img_width, img_height, 3))
model.summary()

#### (VGG19) Freeze the model-layers for transfer-learning and add new classification-layers to the end

In [ ]:
def pop(model):
    '''Removes a layer instance on top of the layer stack.
    '''
    if not model.outputs:
        raise Exception('Sequential model cannot be popped: model is empty.')
    else:
        model.layers.pop()
        if not model.layers:
            model.outputs = []
            model.inbound_nodes = []
            model.outbound_nodes = []
        else:
            model.layers[-1].outbound_nodes = []
            model.outputs = [model.layers[-1].output]
            #model.output = model.outputs[0]
        model.built = False



remove_layers_from_end_amount = 10
for i in range(remove_layers_from_end_amount):
    pop(model)

In [ ]:
model.outputs[0]

In [ ]:
model.output

In [ ]:

x = model.outputs[0]
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.15)(x)
x = Dense(64, activation="relu")(x)

predictions = Dense(int(len(validationGenerator.class_indices)), activation="softmax")(x)

#for layer in model.layers[:6]:
#    layer.trainable = False

model_final = Model(inputs = model.input, outputs = predictions)
model_final.summary()

In [ ]:
model.layers[:12]

#### Initialize Model (Resnet50) Option (A): Transfer Learning

In [ ]:
model = applications.resnet50.ResNet50(input_tensor=Input(shape=(img_width, img_height, 3)), include_top=True,weights='imagenet')

#### (Resnet50) Option (A) Freeze the model-layers for transfer-learning and add one classification-layer

In [ ]:
x = model.get_layer('avg_pool').output
x= Flatten(name='flatten')(x)
predictions = Dense(int(len(validationGenerator.class_indices), activation='softmax')(x)

model_final = Model(inputs=model.input, outputs= predictions)

for layer in model_final.layers[:-1]:
    layer.trainable = False

model_final.layers[-1].trainable = True

model_final.summary()

#### Initialize Model (Resnet50) Option (B): Fine-Tuning

In [ ]:
model = applications.resnet50.ResNet50(weights='imagenet',include_top=False)

#### (Resnet50) Option (B) Freeze the model and replace the "top"

In [ ]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(int(len(validationGenerator.class_indices)), activation='softmax')(x)

model_final = Model(inputs=model.input, outputs=predictions)

#for layer in model_final.layers[:-6]:
#    layer.trainable = False

model_final.layers[-1].trainable = True
model_final.summary()

#### OPTIONAL: Load Model-Weights from saved checkpoint if you already creaded one
You can use this if you had to intercept your training for whatever reason. It makes absolutely no difference, if you load an already trained model or start from scratch

In [ ]:
checkpoint_dir = "crackDetection_vgg19_half.hdf5"
model_final.load_weights(checkpoint_dir)

#### Compile the adjusted Model

In [ ]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.0001), metrics=["accuracy"])

In [ ]:
#model_final.summary()

#### creating Checkpoints for Early-Stopping and to save the best Model

In [ ]:
checkpoint = ModelCheckpoint('./savedModels/crackDetection_vgg19_half_scratchtest.hdf5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=25, verbose=1, mode='auto')


#### Do the Training!

In [ ]:
model_final.fit_generator(
trainGenerator,
steps_per_epoch = int(nb_train_samples/batch_size),
epochs = epochs,
validation_data = validationGenerator,
validation_steps = nb_validation_samples,
callbacks = [checkpoint, early])

In [ ]:
int(nb_train_samples/batch_size)

#### Do some Test-Prediction

In [ ]:
from skimage.transform import resize
TEST_DIR = "data/slices/validation/cracks"
testImages = imageutils.getImageListFromDir(TEST_DIR)
print("Images in Test Folder: {}".format(str(len(testImages))))

In [ ]:
index = random.randint(0,len(testImages)-1)
x = testImages[index]
x = resize(x, (img_width, img_height))
plt.imshow(x)

x = x.reshape((1,img_width,img_height,3))

pred = model_final.predict(x)

print(pred)

cracks = [(item[0], pred[0][item[1]] ) for item in validationGenerator.class_indices.items() if item[1]==np.argmax(pred[0])]
print(cracks)


In [ ]:
validationGenerator.class_indices

'1.10.0'